# Project P3

`Auteur` : Romain Capocasale (INF2dlm-a)

`Date` : 01.10.2019

# Importation

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from sklearn import metrics
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Chragement du jeu de données

In [6]:
dataset = pd.read_json('dataset/financialData.json')

# Analyse du jeu de données
Le jeu de données contient 2 colonnes :
    - sentiment : le sentiment de la pharse, -1 pour positif et 1 pour positif
    - title : la phrase en question

In [86]:
dataset.head()

,sentiment,title
0,0.366,watching for bounce tomorrow
1,0.638,record number of passengers served in 2015
2,-0.494,out $NFLX -.35
3,0.460,Looking for a strong bounce Lunchtime rally co...
4,0.403,Very intrigued with the technology and growth ...


In [87]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 2 columns):
sentiment    2866 non-null float64
title        2866 non-null object
dtypes: float64(1), object(1)
memory usage: 44.9+ KB


In [94]:
dataset.describe()

,sentiment
count,2866.000000
mean,0.079841
std,0.391033
min,-1.000000
25%,-0.285000
50%,0.206000
75%,0.387000
max,1.000000


# Répartition des classes

In [7]:
print("number of positive data : {}".format(len([sent for sent in dataset.sentiment if sent > 0])))
print("number of negative data : {}".format(len([sent for sent in dataset.sentiment if sent < 0])))
print("number of neutral data : {}".format(len([sent for sent in dataset.sentiment if sent == 0])))

number of positive data : 1756
number of negative data : 1045
number of neutral data : 65


On remarque qu'il y a beaucoup plus de phrase avec un sentiment positif que négatif.

# Nombre de mot moyen par phrase

In [8]:
def average_word_by_sentence(sentences):
    nb_total_word = 0
    for row in sentences:
        nb_total_word += len(row.split())
    return round(nb_total_word / len(sentences), 3)

In [9]:
print("average of word by sentence : {}".format(average_word_by_sentence(dataset['title'])) )

average of word by sentence : 7.515


# Récupération des valeurs x et y

In [10]:
X = dataset['title']
y = dataset['sentiment']

# Séparation du jeu d'entrainement et jeu de test 
Je n'utilise pas la classe StratifiedShuffleSplit qui assure une répartition équitable entre le nombre de phrase positive et négative dans le jeu de test et le jeu d'entrainement car étant donné qu'il y a beaucoup plus de phrase positive que négative on perderait une grande partie du jeu de donnée.

In [ ]:
'''sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]'''

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Score de textblob et VADER


In [16]:
y_pred_textblob = []

for x in X_test:
    blob_analyser = TextBlob(x)
    y_pred_textblob.append(blob_analyser.polarity)
    
y_pred_textblob = np.array(y_pred_textblob)
print("RMSE for textblob : {}".format(np.sqrt(metrics.mean_squared_error(y_test, y_pred_textblob))))

RMSE for textblob : 0.3946489568269569


In [17]:
vader_analyser = SentimentIntensityAnalyzer()

y_pred_vader = []
for x in X_test:
    y_pred_vader.append(vader_analyser.polarity_scores(x)['compound'])

y_pred_vader = np.array(y_pred_vader)
print("RMSE for VADER : {}".format(np.sqrt(metrics.mean_squared_error(y_test, y_pred_vader))))

RMSE for VADER : 0.410972061994435
